In [0]:

processName=dbutils.widgets.get("prm_processName")
nextSourceFileDateSql=f"""
SELECT 
MAX(PROCESSED_FILE_TABLE_DATE)+1 AS NEXT_SOURCE_FILE_DATE
FROM
processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS 
WHERE 
PROCESS_NAME = '{processName}' AND PROCESS_STATUS = 'Completed'
"""

print(nextSourceFileDateSql)

nextSourceFileDateDF= spark.sql(nextSourceFileDateSql)
display(nextSourceFileDateDF.select('NEXT_SOURCE_FILE_DATE').collect()[0]['NEXT_SOURCE_FILE_DATE'])

In [0]:
from datetime import datetime
dbutils.widgets.text("prm_dailyPricingSourceFileDate","")
print((datetime.strptime(dbutils.widgets.get("prm_dailyPricingSourceFileDate"),'%Y-%m-%d')).strftime('%d%m%Y'))


In [0]:
sourceFileURL="https://retailpricing.blob.core.windows.net/labs/lab1/PW_MW_DR_01012023.csv"

bronzelayerCSVFilePath= "abfss://testingfiles@datalakestorageaccountname.dfs.core.windows.net/daily-pricing/csv"

In [0]:
dailyPricingSourceBaseURL="https://retailpricing.blob.core.windows.net/"
dailyPricingSourceBaseFolder="daily-pricing/"
dailyPricingSourceFileDate= (datetime.strptime(str(nextSourceFileDateDF.select('NEXT_SOURCE_FILE_DATE').collect()[0]['NEXT_SOURCE_FILE_DATE']),'%Y-%m-%d')).strftime('%d%m%Y')
dailyPricingSourceFileName=f"PW_MW_DR_{dailyPricingSourceFileDate}.csv"

print(dailyPricingSourceFileName)

dailyPricingSinkLayerName = "bronze"
dailyPricingSinkStorageAccountName= "adlsdatalakedeve"
dailyPricingSinkFolderName= "daily-pricing"

In [0]:
import pandas as pds

In [0]:
dailyPricingSourceFileURL= dailyPricingSourceBaseURL+dailyPricingSourceBaseFolder+dailyPricingSourceFileName
print(dailyPricingSourceFileURL)

In [0]:
dailyPricingPandasDF=pds.read_csv(dailyPricingSourceFileURL)
display(dailyPricingPandasDF)

In [0]:
dailyPricingSparkDF= spark.createDataFrame(dailyPricingPandasDF)

In [0]:
dailyPricingSinkFolderPath=f"abfss://{dailyPricingSinkLayerName}@{dailyPricingSinkStorageAccountName}.dfs.core.windows.net/{dailyPricingSinkFolderName}"
print(dailyPricingSinkFolderPath)


In [0]:
dbutils.fs.ls(dailyPricingSinkFolderPath)

In [0]:
(
    dailyPricingSparkDF
    .write
    .format('csv')
    .mode('overwrite')
    .option('header', True)
    .save(dailyPricingSinkFolderPath)

)

In [0]:
spark.read.format('csv').option('header',True).load(dailyPricingSinkFolderPath).display()

In [0]:
%sql
USE CATALOG pricing_analytics;

CREATE SCHEMA IF NOT EXISTS processrunlogs;

CREATE TABLE IF NOT EXISTS processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS(
  PROCESS_NAME STRING,
  PROCESSED_FILE_TABLE_DATE DATE,
  PROCESS_STATUS STRING
)

In [0]:
processName='dailyPricingSourceIngest'
processFileDate = dbutils.widgets.get('prm_dailyPricingSourceFileDate')
processStatus = 'Completed'

processInsertSql = f"""INSERT INTO processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS VALUES('{processName}','{processFileDate}','{processStatus}')"""

spark.sql(processInsertSql)

In [0]:
%sql
SELECT * FROM processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS 

In [0]:
%sql
SELECT 
MAX(PROCESSED_FILE_TABLE_DATE)+1 AS NEXT_SOURCE_FILE_DATE
FROM
processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS 
WHERE 
PROCESS_NAME = 'dailyPricingSourceIngest' AND PROCESS_STATUS = 'Completed'